In this base model ,the code idea has been taken from "LSTM donor choose data".<br>
In this base model what we will do is we will using glove embedding to create embedding layer and also tokenize and pad sequences the text and sentiment column. the target will be selected_text column but it will not be possible to predict selected_text without encoder-decoder model ,but since this is the basic lstm model to predict selected_text so what we will do is we will come up with two new columns for target variable ,these two columns will show the start index and end index of selected_text on text column<br>
for example:

text | selected_text | start_index | end_index |
:---: | :---: | :---: | :---: |
my boss is bullying me | bullying me | 3 | 4|
what interview leave me alone | leave me alone | 2 | 4|

so you can see above "bullying me start from 3rd index in text column and end at 4th index in text column,so  now we will use two newly created columns as target variable and during prediction will replace index with words,<br>This is the **LSTM BASE MODEL**

In [1]:
import warnings
warnings.filterwarnings("ignore")
import re
import os
import random
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy.stats as st
import matplotlib as mpl
import matplotlib.patches as mpl_patches
import matplotlib.cm as cm
import matplotlib.lines as mlines
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')  
nltk.download('vader_lexicon')
import spacy
from collections import defaultdict,Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer  

from textstat import flesch_reading_ease
tqdm.pandas()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,GRU,LSTM
from tensorflow.keras.models import Model
import keras.backend as K
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding,Dense,Dropout,Concatenate,Flatten,Input,GRU,BatchNormalization,Bidirectional,SpatialDropout1D,LSTM,LayerNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint,EarlyStopping,TensorBoard,ReduceLROnPlateau

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
df_train = pd.read_csv('train_Preprocessed_final.csv')
df_train.head(2)

,textID,text,selected_text,sentiment
0,cb774db0d1,i`d have responded if i were going,i`d have responded if i were going,neutral
1,549e992a42,sooo sad i will miss you here in san diego,sooo sad,negative


In [3]:
df_test = pd.read_csv('test_preprocessed_final.csv')
df_test.head(2)

,textID,text,sentiment
0,f87dea47db,last session of the day,neutral
1,96d74cb729,shanghai is also really exciting precisely sky...,positive


In [4]:
print('The training data has a shape of : ',df_train.shape)
print('The test data has a shape of : ',df_test.shape)

The training data has a shape of :  (27468, 4)
The test data has a shape of :  (3533, 3)


In [ ]:
def start_index(x):
    '''This function will find the starting index from text column where words from selected_text starts
    text == attribute1
    selected_text == attribute2'''
    attribute1 = data[0]
    attribute2 = data[1]
    attribute1 = attribute1.split()
    attribute2 = attribute2.split()
    end = attribute2[0]
    starting_index = attribute1.index(end)
    return starting_index

In [ ]:
def end_index(x):
    '''This function will find the ending index from text column where words from selected_text starts
    text == attribute1
    selected_text == attribute2
    start_index == attribute3'''
    attribute1 = data[0]
    attribute2 = data[1]
    attribute3 = data[2]
    attribute2 = attribute2.split()
    end = attribute2[-1]
    try:
        ending_index = attribute1.index(end,attribute3)
    except:
        ending_index = attribute1.index(end)
        
    return ending_index

In [7]:
df_train['starting_index'] = df_train[['text','selected_text']].progress_apply(lambda i : start_index(i),axis=1)

100%|█████████████████████████████████████████████████████████████████████████| 27468/27468 [00:00<00:00, 43156.65it/s]


In [8]:
df_train['ending_index'] = df_train[['text','selected_text','starting_index']].progress_apply(lambda i : end_index(i),axis=1)

100%|█████████████████████████████████████████████████████████████████████████| 27468/27468 [00:00<00:00, 32982.76it/s]


#### checking whether ending index < starting index and if yes removing that row

In [9]:
df_train[df_train.ending_index<df_train.starting_index].shape

(23, 6)

In [10]:
# we will take only those rows where ending_index >= starting index
df_train = df_train[df_train.ending_index >= df_train.starting_index]

In [11]:
df_train.shape

(27445, 6)

In [12]:
df_train.head()

,textID,text,selected_text,sentiment,starting_index,ending_index
0,cb774db0d1,i`d have responded if i were going,i`d have responded if i were going,neutral,0,6
1,549e992a42,sooo sad i will miss you here in san diego,sooo sad,negative,0,1
2,088c60f138,my boss is bullying me,bullying me,negative,3,4
3,9642c003ef,what interview leave me alone,leave me alone,negative,2,4
4,358bd9e861,sons of CURSE why couldn`t they put them on th...,sons of CURSE,negative,0,2


## Train Validation Split

In [ ]:
#df_train.to_csv('final_train.csv')

In [13]:
X = df_train[['text','sentiment','selected_text']]
y = df_train[['starting_index','ending_index']]

In [14]:
print('The shape of X is :',X.shape)
print('The shape of y is :',y.shape)

The shape of X is : (27445, 3)
The shape of y is : (27445, 2)


In [15]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid= train_test_split(X,y,test_size=0.20,random_state=42)

print('the shape of X_train is : ',X_train.shape)
print('the shape of X_valid is : ',X_valid.shape)
print('the shape of y_train is : ',y_train.shape)
print('the shape of y_valid is : ',y_valid.shape)

the shape of X_train is :  (21956, 3)
the shape of X_valid is :  (5489, 3)
the shape of y_train is :  (21956, 2)
the shape of y_valid is :  (5489, 2)


In [16]:
train_new = X_train.copy()
train_new.shape

(21956, 3)

In [17]:
train_new.head(2)

,text,sentiment,selected_text
9772,someone take me to la i need to see the lemon ...,neutral,someone take me to la i need to see the lemon ...
26215,lol heeeyyy love not much working and you,neutral,lol heeeyyy love not much working and you


In [18]:
valid_new = X_valid.copy()
valid_new.shape

(5489, 3)

In [19]:
valid_new.head(2)

,text,sentiment,selected_text
1366,sooo glad im home floridia was fun back in atl...,positive,glad
11297,csi ny comes back to aus with ` CURSE lies and...,neutral,csi ny comes back to aus with ` CURSE lies and...


## "Text" Column

In [20]:
df_text_train = X_train['text'].values
df_text_valid = X_valid['text'].values

#### Tokenizer and pad sequencing

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
text_tknz = Tokenizer(lower=True,split=' ',oov_token='oov')
text_tknz.fit_on_texts(df_text_train)
df_text_train = text_tknz.texts_to_sequences(df_text_train)
df_text_valid = text_tknz.texts_to_sequences(df_text_valid)
text_vocab_size = len(text_tknz.word_index)+1
print(text_vocab_size)

20710


In [23]:
max_length = 40
from tensorflow.keras.preprocessing.sequence import pad_sequences
df_text_train = pad_sequences(df_text_train,maxlen=max_length,padding='post')
df_text_valid = pad_sequences(df_text_valid,maxlen=max_length,padding='post')

print('the shape of text train after padding : ',df_text_train.shape)
print('the shape of text valid after padding : ',df_text_valid.shape)

the shape of text train after padding :  (21956, 40)
the shape of text valid after padding :  (5489, 40)


#### Embedding Matrix

In [24]:
from numpy import asarray
from numpy import zeros
embeddings_index = dict()
with open('glove.6B.300dd.txt',encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    
print(len(embeddings_index))

400000


In [26]:
embedding_matrix = zeros((text_vocab_size, 300))
for word, i in text_tknz.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

(20710, 300)


## "Sentiment" Column

In [28]:
df_sentiment_train = X_train['sentiment'].values
df_sentiment_valid = X_valid['sentiment'].values

#### Tokenizer and Pad Sequencing

In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer
sentiment_tknz = Tokenizer(lower=True,split=' ',oov_token='oov')
sentiment_tknz.fit_on_texts(df_sentiment_train)
df_sentiment_train = sentiment_tknz.texts_to_sequences(df_sentiment_train)
df_sentiment_valid = sentiment_tknz.texts_to_sequences(df_sentiment_valid)
sentiment_vocab_size = len(sentiment_tknz.word_index)+1
print(sentiment_vocab_size)

5


In [30]:
max_length1 = 1
from tensorflow.keras.preprocessing.sequence import pad_sequences
df_sentiment_train = pad_sequences(df_sentiment_train,maxlen=max_length1,padding='post')
df_sentiment_valid = pad_sequences(df_sentiment_valid,maxlen=max_length1,padding='post')

print('the shape of sentiment train after padding : ',df_sentiment_train.shape)
print('the shape of sentiment valid after padding : ',df_sentiment_valid.shape)

the shape of sentiment train after padding :  (21956, 1)
the shape of sentiment valid after padding :  (5489, 1)


In [28]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding,Dense,Dropout,Concatenate,Flatten,Input,GRU,BatchNormalization,Bidirectional,SpatialDropout1D,LSTM,LayerNormalization
from tensorflow.keras.regularizers import l2

## LSTM Model

In [29]:
## text column

text_input=Input(shape=(max_length,))
embedding_text = Embedding(text_vocab_size,300,input_length=max_length,\
                      trainable=False,embeddings_initializer=tf.constant_initializer(embedding_matrix))(text_input)
lstm_layer_text=LSTM(100,return_sequences=True)(embedding_text)
flatten_text=Flatten()(lstm_layer_text)

## Sentiment column

sentiment_input=Input(shape=(max_length1,))
embedding_sentiment=Embedding(sentiment_vocab_size,10,input_length=max_length1,name='embedding_sentiment')(sentiment_input)
flatten_sentiment=Flatten()(embedding_sentiment)

#concatenating
concat=Concatenate(axis=1)([flatten_text,flatten_sentiment])
dense1=Dense(32,activation='relu',kernel_regularizer=l2(0.0001))(concat)
dropout = Dropout(0.6)(dense1)
layer_n= LayerNormalization()(dropout)
dense2=Dense(16,activation='relu',kernel_regularizer=l2(0.0001))(layer_n)
output=Dense(1,name='output')(dense2)


In [32]:
model=Model(inputs=[text_input,sentiment_input],outputs=[output])

In [33]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_text (InputLayer)         [(None, 40)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 40, 300)      6213000     input_text[0][0]                 
__________________________________________________________________________________________________
input_sentiment (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
GRU (GRU)                       (None, 40, 32)       32064       embedding[0][0]                  
______________________________________________________________________________________________

In [34]:
train_input = (df_text_train, df_sentiment_train)
train_output = y_train.values

valid_input = (df_text_valid,df_sentiment_valid)
valid_output = y_valid.values

In [35]:
import datetime
import os
! mkdir  'checkpt1'
file_path = os.path.join('model11.hdf5')
earlystop = EarlyStopping(monitor='val_mae', min_delta=0.001, patience=10, verbose=1,mode='min')

reducelr = ReduceLROnPlateau(monitor='val_mae', min_delta=0.001, patience=5, verbose=1, factor=0.9)
checkpt_model = tf.keras.callbacks.ModelCheckpoint(filepath=file_path,save_weights_only=True,monitor='val_mae',save_best_only=True,verbose=1)
callbacks=[checkpt_model,earlystop,reducelr]

A subdirectory or file 'checkpt1' already exists.


In [36]:
model.compile(optimizer='adam',loss="mse",metrics=["mae"])

In [37]:
model.fit(train_input,train_output,epochs=300,batch_size=128,validation_data=(valid_input,valid_output),validation_batch_size=64,callbacks=callbacks)

Epoch 1/300
172/172 [==============================] - 8s 27ms/step - loss: 37.7273 - mae: 4.3495 - val_loss: 25.8844 - val_mae: 3.5938

Epoch 00001: val_mae improved from inf to 3.59384, saving model to model11.hdf5
Epoch 2/300
172/172 [==============================] - 4s 24ms/step - loss: 25.6899 - mae: 3.7032 - val_loss: 23.9546 - val_mae: 3.3679

Epoch 00002: val_mae improved from 3.59384 to 3.36793, saving model to model11.hdf5
Epoch 3/300
172/172 [==============================] - 4s 24ms/step - loss: 22.4410 - mae: 3.4431 - val_loss: 17.1379 - val_mae: 2.9220

Epoch 00003: val_mae improved from 3.36793 to 2.92200, saving model to model11.hdf5
Epoch 4/300
172/172 [==============================] - 4s 24ms/step - loss: 20.3695 - mae: 3.2374 - val_loss: 19.0331 - val_mae: 2.9653

Epoch 00004: val_mae did not improve from 2.92200
Epoch 5/300
172/172 [==============================] - 4s 24ms/step - loss: 19.6897 - mae: 3.1584 - val_loss: 15.6808 - val_mae: 2.7675

Epoch 00005: val_

In [38]:
model.load_weights(file_path)

## Jaccard-Score on Train Data

In [39]:
y_prediction = model.predict(input_data)
y_true = output_data

In [40]:
y_prediction

array([[-0.17737427, 11.779454  ],
       [ 0.8059147 ,  4.1807423 ],
       [ 2.0188372 ,  3.5744896 ],
       ...,
       [ 1.3188293 ,  4.244046  ],
       [ 6.375878  ,  9.905171  ],
       [ 1.8609613 ,  3.46247   ]], dtype=float32)

In [41]:
y_prediction.shape

(21956, 2)

In [42]:
y_prediction = np.round(y_prediction)
y_prediction

array([[-0., 12.],
       [ 1.,  4.],
       [ 2.,  4.],
       ...,
       [ 1.,  4.],
       [ 6., 10.],
       [ 2.,  3.]], dtype=float32)

In [43]:
train_new['start'],train_new['end'] = (abs(y_prediction[:,0])),(abs(y_prediction[:,1]))

In [44]:
train_new['start'] = train_new['start'].astype('int')
train_new['end'] = train_new['end'].astype('int')

In [45]:
def converting_index_to_text(data):
    """This function reconvert the starting and ending index back to text"""
  
     text_col = data[0]
    
     start_col = data[1]
     
     end_col = data[2]
    
     end_col =end_col + 1
    
     text_col = text_col.split()
    
     selected_text_col = text_col[start_col:end_col]
    
     selected_text_col = " ".join(selected_text_col)
 
     return selected_text_col

In [46]:
train_new['predicted_text'] = train_new[['text','start','end']].progress_apply(lambda i : converting_index_to_text(i),axis=1)

100%|█████████████████████████████████████████████████████████████████████████| 21956/21956 [00:00<00:00, 57364.70it/s]


In [47]:
train_new.head()

,text,sentiment,selected_text,start,end,predicted_text
9772,someone take me to la i need to see the lemon ...,neutral,someone take me to la i need to see the lemon ...,0,12,someone take me to la i need to see the lemon ...
26215,lol heeeyyy love not much working and you,neutral,lol heeeyyy love not much working and you,1,4,heeeyyy love not much
12647,can`t wait to see `transformers ` c`me on shia...,positive,yippiee,2,4,to see `transformers
10567,but then you won`t see her hotness much either,neutral,but then you won`t see her hotness much either,0,8,but then you won`t see her hotness much either
14913,hi mel if you feel unhappy just take a bit of ...,negative,unhappy,7,10,take a bit of


In [48]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [49]:
true_values = train_new['selected_text'].values
predicted_values = train_new['predicted_text'].values

In [50]:
scores=[]
for i in tqdm(range(len(true_values))):
  scores.append(jaccard(true_values[i],predicted_values[i]))

100%|████████████████████████████████████████████████████████████████████████| 21956/21956 [00:00<00:00, 196845.72it/s]


In [51]:
train_new['jaccard_score'] = scores
train_new.sample(15)

,text,sentiment,selected_text,start,end,predicted_text,jaccard_score
22026,its awful xxxx,negative,its awful,1,3,awful xxxx,0.333333
10282,hahaha i think you`re pretty far from me maybe...,neutral,hahaha i think you`re pretty far from me,1,12,i think you`re pretty far from me maybe next t...,0.583333
11704,pfft there you go with the logic again,neutral,pfft there you go with the logic again,1,6,there you go with the logic,0.750000
6755,going to look at a bunch of open homes today j...,neutral,going to look at a bunch of open homes today j...,2,24,look at a bunch of open homes today just to ge...,0.869565
10531,gettin my cardio on right now walkin to the tr...,negative,i so miss the gym,10,14,station awww i so miss,0.428571
3303,unfortunately no,negative,unfortunately,1,3,no,0.000000
25043,who was it that said vive le difference to me ...,neutral,who was it that said vive le difference to me ...,0,14,who was it that said vive le difference to me ...,1.000000
17365,hey handsome i`m packing,positive,handsome,2,3,i`m packing,0.000000
21249,very disappointed in him gonna have to sleep i...,negative,disappointed,2,3,in him,0.000000
22134,i`m probably the only person in the world that...,neutral,i`m probably the only person in the world that...,1,24,probably the only person in the world that use...,0.947368


In [52]:
print('Mean jaccard score for training data:', np.mean(train_new['jaccard_score']))

Mean jaccard score for training data: 0.5037036122280315


## Jaccard_score on Validation data

In [53]:
y_prediction_valid = model.predict(val)
y_true_valid = output_val

In [54]:
y_prediction_valid.shape

(5489, 2)

In [55]:
valid_new['start'],valid_new['end'] = (abs(y_prediction_valid[:,0])),(abs(y_prediction_valid[:,1]))

In [56]:
valid_new['start'] = valid_new['start'].astype('int')
valid_new['end'] = valid_new['end'].astype('int')

In [57]:
valid_new['predicted_text'] = valid_new[['text','start','end']].progress_apply(lambda i : converting_index_to_text(i),axis=1)

100%|███████████████████████████████████████████████████████████████████████████| 5489/5489 [00:00<00:00, 61286.71it/s]


In [58]:
valid_new.head()

,text,sentiment,selected_text,start,end,predicted_text
1366,sooo glad im home floridia was fun back in atl...,positive,glad,1,2,glad im
11297,csi ny comes back to aus with ` CURSE lies and...,neutral,csi ny comes back to aus with ` CURSE lies and...,0,16,csi ny comes back to aus with ` CURSE lies and...
16029,got so totally lost that i had to pay for a ta...,negative,lost,2,1,
23965,that`s how it is supposed to be and sometimes ...,neutral,that`s how it is supposed to be and sometimes ...,1,22,how it is supposed to be and sometimes we get ...
4639,out with the guys playing pin doing mexican gr...,neutral,out with the guys playing pin doing mexican gr...,0,26,out with the guys playing pin doing mexican gr...


In [59]:
true_values1 = valid_new['selected_text'].values
predicted_values1 = valid_new['predicted_text'].values

In [60]:
scores=[]
for i in tqdm(range(len(true_values1))):
  scores.append(jaccard(true_values1[i],predicted_values1[i]))

100%|██████████████████████████████████████████████████████████████████████████| 5489/5489 [00:00<00:00, 141128.26it/s]


In [61]:
valid_new['jaccard_score'] = scores
valid_new.sample(5)

,text,sentiment,selected_text,start,end,predicted_text,jaccard_score
14976,g`mornin,positive,g`mornin,1,2,,0.000000
2981,im so sick i fell over last night outside coz ...,negative,im so sick,0,1,im so,0.666667
25479,it`s so hard to keep up with everything there`...,negative,so hard,2,5,hard to keep up,0.200000
13761,aww no one`s on msn,neutral,aww no one`s on msn,1,3,no one`s on,0.600000
11341,diva i know i know lol that is the one song th...,neutral,diva i know i know lol that is the one song th...,0,27,diva i know i know lol that is the one song th...,1.000000


In [62]:
print('Mean jaccard score for validation data:', np.mean(valid_new['jaccard_score']))

Mean jaccard score for validation data: 0.49236118358691255


## Prediction on Test Data

In [31]:
df_test.head()

,textID,text,sentiment
0,f87dea47db,last session of the day,neutral
1,96d74cb729,shanghai is also really exciting precisely sky...,positive
2,eee518ae67,recession hit veronique branquinho she has to ...,negative
3,01082688c6,happy bday,positive
4,33987a8ee5,i like it,positive


#### Tokenizing and pad sequencing Text Column

In [32]:
df_text_test = df_test['text'].values
df_text_test = text_tknz.texts_to_sequences(df_text_test)
df_text_test = pad_sequences(df_text_test,maxlen=max_length,padding='post')
print('the shape of test text : ',df_text_test.shape)
print('the shape of test text : ',(3533, 40))

the shape of test text :  (3533, 40)


#### Tokenizing and pad sequencing sentiment column

In [34]:
df_sentiment_test = df_test['sentiment'].values
df_sentiment_test = sentiment_tknz.texts_to_sequences(df_sentiment_test)
df_sentiment_test = pad_sequences(df_sentiment_test,maxlen=max_length1,padding='post')
print('the shape of test sentiment : ',df_sentiment_test.shape)

the shape of test sentiment :  (3533, 1)


In [66]:
predicted_results  = model.predict([test_text,test_sentiment])

In [67]:
predicted_results.shape

(3533, 2)

In [68]:
predicted_results = np.round(predicted_results)
predicted_results

array([[ 1.,  3.],
       [ 2.,  4.],
       [ 9., 13.],
       ...,
       [ 6., 10.],
       [ 9., 13.],
       [ 1.,  3.]], dtype=float32)

In [69]:
df_test['start'] = abs(predicted_results[:,0])
df_test['end'] = abs(predicted_results[:,1])

In [70]:
df_test['start'] = df_test['start'].astype('int')
df_test['end'] = df_test['end'].astype('int')

In [71]:
df_test

,textID,text,sentiment,start,end
0,f87dea47db,last session of the day,neutral,1,3
1,96d74cb729,shanghai is also really exciting precisely sky...,positive,2,4
2,eee518ae67,recession hit veronique branquinho she has to ...,negative,9,13
3,01082688c6,happy bday,positive,1,2
4,33987a8ee5,i like it,positive,1,3
...,...,...,...,...,...
3528,e5f0e6ef4b,its at am im very tired but i can`t sleep but ...,negative,3,4
3529,416863ce47,all alone in this old house again thanks for t...,positive,11,15
3530,6332da480c,i know what you mean my little dog is sinking ...,negative,6,10
3531,df1baec676,sutra what is your next youtube video gonna be...,positive,9,13


In [72]:
df_test['selected_text'] = df_test[['text','start','end']].progress_apply(lambda i : converting_index_to_text(i),axis=1)

100%|███████████████████████████████████████████████████████████████████████████| 3533/3533 [00:00<00:00, 47229.90it/s]


In [73]:
df_test

,textID,text,sentiment,start,end,selected_text
0,f87dea47db,last session of the day,neutral,1,3,session of the
1,96d74cb729,shanghai is also really exciting precisely sky...,positive,2,4,also really exciting
2,eee518ae67,recession hit veronique branquinho she has to ...,negative,9,13,company such a shame
3,01082688c6,happy bday,positive,1,2,bday
4,33987a8ee5,i like it,positive,1,3,like it
...,...,...,...,...,...,...
3528,e5f0e6ef4b,its at am im very tired but i can`t sleep but ...,negative,3,4,im very
3529,416863ce47,all alone in this old house again thanks for t...,positive,11,15,which keeps me alive and
3530,6332da480c,i know what you mean my little dog is sinking ...,negative,6,10,little dog is sinking into
3531,df1baec676,sutra what is your next youtube video gonna be...,positive,9,13,about i love your videos


## Conclusion:
we can see the jaccard similarityity for train data is 0.5037 and for validation data is 0.49,and thus came to conclusion that the basic lstm model is not performing well.So we have to use the advance model with lstm with attention mechanism or bert model to see whether we are getting good result.